In [1]:
import pandas as pd
import os

def clean_battery_data():
    # Find all Excel files in the current directory
    files = [f for f in os.listdir('.') if f.endswith(('.xlsx', '.xls'))]
    
    if not files:
        print("No Excel files found.")
        return

    print(f"Found {len(files)} files. Processing...\n")

    for file in files:
        # Skip files we've already generated
        if file.startswith("Cleaned_"):
            continue

        try:
            print(f"Processing: {file}...")
            
            # Load the Excel file
            xls = pd.ExcelFile(file)
            processed_sheets = {}

            for sheet_name in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet_name)
                
                # Clean up whitespace in column names just in case
                df.columns = df.columns.str.strip()

                # --- CONFIGURATION ---
                # Pandas handles duplicate columns by adding .1, .2, etc.
                # Col B: 'control/V' (Original) -> We keep this for filtering
                # Col G: 'control/V.1' (Duplicate) -> We want to delete this
                
                col_vol_keep = 'control/V'      # Matches Column B
                col_current  = '<I>/mA'         # Matches Column D
                col_ctrl_ma  = 'control/mA'     # Matches Column H
                col_to_drop  = 'control/V.1'    # Matches Column G (The duplicate)

                # --- STEP 1: Remove rows where all 3 specific columns are 0 ---
                # We check if the necessary columns exist
                if {col_vol_keep, col_current, col_ctrl_ma}.issubset(df.columns):
                    
                    # Create the filter condition (All 3 must be 0)
                    condition_mask = (
                        (df[col_vol_keep] == 0) & 
                        (df[col_current] == 0) & 
                        (df[col_ctrl_ma] == 0)
                    )
                    
                    # Keep rows that do NOT match the condition
                    df = df[~condition_mask]
                else:
                    print(f"  - Sheet '{sheet_name}': Could not find all filter columns. Skipping row removal.")

                # --- STEP 2: Delete ONLY the duplicate column (Col G) ---
                if col_to_drop in df.columns:
                    df = df.drop(columns=[col_to_drop])
                    # print(f"  - Sheet '{sheet_name}': Dropped duplicate column '{col_to_drop}'.")
                
                # Add processed dataframe to our dictionary
                processed_sheets[sheet_name] = df

            # --- STEP 3: Save to new file ---
            output_filename = f"Cleaned_{file}"
            
            # Using openpyxl engine for .xlsx files
            with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
                for sheet_name, data in processed_sheets.items():
                    data.to_excel(writer, sheet_name=sheet_name, index=False)
            
            print(f"Done. Saved as: {output_filename}\n")

        except Exception as e:
            print(f"Error processing {file}: {e}\n")

if __name__ == "__main__":
    clean_battery_data()

Found 9 files. Processing...

Processing: CY25-05_1-#1.xlsx...
Done. Saved as: Cleaned_CY25-05_1-#1.xlsx

Processing: CY25-05_1-#2.xlsx...
Done. Saved as: Cleaned_CY25-05_1-#2.xlsx

Processing: CY25-05_1-#3.xlsx...
Error processing CY25-05_1-#3.xlsx: File is not a zip file

Processing: CY25-05_2-#1.xlsx...
Done. Saved as: Cleaned_CY25-05_2-#1.xlsx

Processing: CY25-05_2-#2.xlsx...
Done. Saved as: Cleaned_CY25-05_2-#2.xlsx

Processing: CY25-05_2-#3.xlsx...
Done. Saved as: Cleaned_CY25-05_2-#3.xlsx

Processing: CY25-05_4-#1.xlsx...
Done. Saved as: Cleaned_CY25-05_4-#1.xlsx

Processing: CY25-05_4-#2.xlsx...
Done. Saved as: Cleaned_CY25-05_4-#2.xlsx

Processing: CY25-05_4-#3.xlsx...
Done. Saved as: Cleaned_CY25-05_4-#3.xlsx



In [2]:
import pandas as pd
import numpy as np
import os

def delete_spike_rows(df, col_name, window=20, threshold=3):
    """
    Identifies spikes using Z-score and DELETES the entire row.
    """
    # Calculate Rolling Stats
    rolling_mean = df[col_name].rolling(window=window, center=True).mean()
    rolling_std = df[col_name].rolling(window=window, center=True).std()

    # Detect Outliers (Spikes)
    # A point is an outlier if it deviates significantly from the local average
    is_outlier = (df[col_name] - rolling_mean).abs() > (threshold * rolling_std)

    # Count how many we are dropping
    num_outliers = is_outlier.sum()
    
    if num_outliers > 0:
        # Keep only the rows that are NOT outliers
        df_clean = df[~is_outlier].copy()
        print(f"    - Found and deleted {num_outliers} spike/outlier rows based on '{col_name}'.")
        return df_clean
    else:
        return df

def process_battery_data():
    files = [f for f in os.listdir('.') if f.endswith(('.xlsx', '.xls'))]
    
    # Ignore output files to prevent re-processing
    files = [f for f in files if not f.startswith("spikeremoved_")]

    if not files:
        print("No Excel files found.")
        return

    print(f"Found {len(files)} files. Processing...\n")

    for file in files:
        try:
            print(f"Processing: {file}...")
            xls = pd.ExcelFile(file)
            processed_sheets = {}

            for sheet_name in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet_name)
                df.columns = df.columns.str.strip()

                # --- CONFIGURATION ---
                col_vol     = 'control/V'      # For filtering
                col_cur     = '<I>/mA'         # For filtering 0s
                col_ctrl_ma = 'control/mA'     # For filtering 0s
                col_ecell   = 'Ecell/V'        # For detecting spikes
                
                target_val  = 4.1999354        # The exact value to remove

                # 1. Remove "All Zero" Rows (control/V, I, control/mA are all 0)
                if {col_vol, col_cur, col_ctrl_ma}.issubset(df.columns):
                    condition_zeros = (
                        (df[col_vol] == 0) & 
                        (df[col_cur] == 0) & 
                        (df[col_ctrl_ma] == 0)
                    )
                    df = df[~condition_zeros]

                # 2. Remove rows where control/V is exactly 4.1999354
                if col_vol in df.columns and pd.api.types.is_numeric_dtype(df[col_vol]):
                    # We use isclose to handle floating point precision safely
                    # This finds rows equal to 4.1999354
                    is_target = np.isclose(df[col_vol], target_val, atol=1e-6)
                    if is_target.sum() > 0:
                        print(f"    - Sheet '{sheet_name}': Deleted {is_target.sum()} rows where {col_vol} == {target_val}")
                        df = df[~is_target]

                # 3. Delete Outlier Rows (Spikes) in Ecell/V
                if col_ecell in df.columns:
                    # Note: You can adjust 'threshold' (default 3) if it deletes too much/too little
                    df = delete_spike_rows(df, col_ecell, window=20, threshold=3)

                # Store the cleaned data
                processed_sheets[sheet_name] = df

            # --- Save to new file ---
            output_filename = f"spikeremoved_{file}"
            with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
                for sheet_name, data in processed_sheets.items():
                    data.to_excel(writer, sheet_name=sheet_name, index=False)
            
            print(f"Saved: {output_filename}\n")

        except Exception as e:
            print(f"Error processing {file}: {e}\n")

if __name__ == "__main__":
    process_battery_data()

Found 9 files. Processing...

Processing: Cleaned_CY25-05_1-#1.xlsx...
    - Sheet 'Sheet_1': Deleted 131905 rows where control/V == 4.1999354
    - Sheet 'Sheet_2': Deleted 215027 rows where control/V == 4.1999354
    - Sheet 'Sheet_3': Deleted 251356 rows where control/V == 4.1999354
Saved: spikeremoved_Cleaned_CY25-05_1-#1.xlsx

Processing: Cleaned_CY25-05_1-#2.xlsx...
    - Sheet 'Sheet_1': Deleted 130671 rows where control/V == 4.1999354
    - Sheet 'Sheet_2': Deleted 214404 rows where control/V == 4.1999354
    - Sheet 'Sheet_3': Deleted 259999 rows where control/V == 4.1999354
    - Sheet 'Sheet_4': Deleted 941 rows where control/V == 4.1999354
Saved: spikeremoved_Cleaned_CY25-05_1-#2.xlsx

Processing: Cleaned_CY25-05_2-#1.xlsx...
    - Sheet 'Sheet_1': Deleted 156881 rows where control/V == 4.1999354
    - Sheet 'Sheet_2': Deleted 245407 rows where control/V == 4.1999354
    - Sheet 'Sheet_3': Deleted 227911 rows where control/V == 4.1999354
    - Found and deleted 2 spike/outl